In [27]:
import os, time, sqlite3, requests

# ── 設定 ────────────────────────────────────────────────
ORG  = "google"            # 対象 Organization
PER  = 100                 # 1 ページ取得件数 (API 上限 100)
DB   = "google_repos.db"   # 保存先 DB
TOKEN = os.getenv("GH_TOKEN")            # アクセストークン(任意だが推奨)

headers = {"Authorization": f"token {TOKEN}"} if TOKEN else {}
# ──────────────────────────────────────────────────────

# ── DB 準備 ────────────────────────────────────────────
conn = sqlite3.connect(DB)
cur  = conn.cursor()
cur.execute("""
CREATE TABLE IF NOT EXISTS repos(
    id       INTEGER PRIMARY KEY AUTOINCREMENT,
    name     TEXT UNIQUE,
    language TEXT,
    stars    INTEGER
)""")
conn.commit()

# ── API で取得・保存 ──────────────────────────────────
page = 1
while True:
    url = f"https://api.github.com/orgs/{ORG}/repos?per_page={PER}&page={page}"
    print("[+] GET", url)
    resp = requests.get(url, headers=headers, timeout=40)
    resp.raise_for_status()
    data = resp.json()
    if not data:          # 空になったら終了
        break

    for repo in data:
        cur.execute(
            "INSERT OR REPLACE INTO repos(name, language, stars) VALUES (?,?,?)",
            (repo["name"], repo["language"], repo["stargazers_count"])
        )
    conn.commit()

    page += 1
    time.sleep(1)         # polite (rate-limit 対策)

print("\n=== crawl finished ===")
total = cur.execute("SELECT COUNT(*) FROM repos").fetchone()[0]
print(f"Total repos stored: {total}\n")

# ── 表示 ───────────────────────────────────────────────
print(f"{'Repository':30} | {'Language':10} | {'Stars':>7}")
print("-"*55)
for name, lang, stars in cur.execute(
        "SELECT name, language, stars FROM repos ORDER BY stars DESC"):
    print(f"{name:30} | {lang or 'N/A':10} | {stars:7}")

conn.close()

[+] GET https://api.github.com/orgs/google/repos?per_page=100&page=1
[+] GET https://api.github.com/orgs/google/repos?per_page=100&page=2
[+] GET https://api.github.com/orgs/google/repos?per_page=100&page=3
[+] GET https://api.github.com/orgs/google/repos?per_page=100&page=4
[+] GET https://api.github.com/orgs/google/repos?per_page=100&page=5
[+] GET https://api.github.com/orgs/google/repos?per_page=100&page=6
[+] GET https://api.github.com/orgs/google/repos?per_page=100&page=7
[+] GET https://api.github.com/orgs/google/repos?per_page=100&page=8
[+] GET https://api.github.com/orgs/google/repos?per_page=100&page=9
[+] GET https://api.github.com/orgs/google/repos?per_page=100&page=10
[+] GET https://api.github.com/orgs/google/repos?per_page=100&page=11
[+] GET https://api.github.com/orgs/google/repos?per_page=100&page=12
[+] GET https://api.github.com/orgs/google/repos?per_page=100&page=13
[+] GET https://api.github.com/orgs/google/repos?per_page=100&page=14
[+] GET https://api.github.co